In [1]:
import pandas as pd
import os
import json
from typing import List

In [2]:
ATTACK_NUMBER = 'attack_no'
ENTRY_RESULTS = 'entry_results'
RESULT_PATH = '../results/2022-04-18 17.33.27.936980.json'

In [3]:
def entry_result_selector(key: str, l: int) -> List[str]:
    return [f'{ENTRY_RESULTS}.{i}.{key}' for i in range(l)]
    

In [4]:
def list_to_dict(l: list) -> dict:
    d = {}
    for i, value in enumerate(l):
        d[i] = value
    return d

In [5]:
def load_results(result_path: str) -> List[dict]:    
    with open(result_path, 'r') as f:
        results = json.load(f)
    
    for i, result in enumerate(results):
        result[ATTACK_NUMBER] = i
        result[ENTRY_RESULTS] = list_to_dict(result[ENTRY_RESULTS])
    
    return results, len(results[0][ENTRY_RESULTS])

In [6]:
def aggregate_values(df: pd.DataFrame, l: int) -> pd.DataFrame:
    for i in range(l):
        prefix = f'{ENTRY_RESULTS}.{i}'
        
        df[f'{prefix}.false_positive_rate'] = (df[f'{prefix}.filtered_equations'] - df[f'{prefix}.faulted_coeffs']) / df[f'{prefix}.filtered_equations']
    
    df['duration'] = df[entry_result_selector('duration', l)].sum(axis=1)
    df['false_positive_rate'] = df[entry_result_selector('false_positive_rate', l)].mean(axis=1)
    df['success'] = (df[entry_result_selector('failure', l)] == False).all(axis=1)
    
 
    return df

In [7]:
results, l = load_results(RESULT_PATH)
data = pd.json_normalize(results)
data = data.set_index([ATTACK_NUMBER])
data

,m,num_signatures,nist_param_level,threshold,entry_results.0.faulted_coeffs,entry_results.0.total_equations,entry_results.0.filtered_equations,entry_results.0.timeout_limit,entry_results.0.duration,entry_results.0.failure,...,entry_results.4.filtered_equations,entry_results.4.timeout_limit,entry_results.4.duration,entry_results.4.failure,entry_results.4.equations_used,entry_results.1.failure_reason,entry_results.2.failure_reason,entry_results.3.failure_reason,entry_results.4.failure_reason,entry_results.0.failure_reason
attack_no,,,,,,,,,,,,,,,,,,,,,
0,1270,131,3,196,257,33536,265,60,0.268868,False,...,276,60.0,0.416339,False,262.0,NaN,NaN,NaN,NaN,NaN
1,1270,135,3,196,257,34560,266,60,0.199509,False,...,295,60.0,0.311218,False,281.0,NaN,NaN,NaN,NaN,NaN
2,1270,135,3,196,279,34560,288,60,0.208690,False,...,272,60.0,0.386750,False,259.0,NaN,NaN,NaN,NaN,NaN
3,1271,154,3,196,286,39424,297,60,0.248354,False,...,299,60.0,0.285979,False,284.0,NaN,NaN,NaN,NaN,NaN
4,1271,155,3,196,257,39680,266,60,0.240189,False,...,299,60.0,0.411673,False,284.0,NaN,NaN,NaN,NaN,NaN
5,1271,153,3,196,280,39168,293,60,0.241885,False,...,305,60.0,0.311165,False,286.0,NaN,NaN,NaN,NaN,NaN
6,1272,180,3,196,318,46080,335,60,0.368590,False,...,268,60.0,5.183606,False,257.0,NaN,NaN,NaN,NaN,NaN
7,1272,169,3,196,263,43264,280,60,0.311557,False,...,282,60.0,0.275138,False,270.0,NaN,NaN,NaN,NaN,NaN
8,1272,169,3,196,271,43264,285,60,0.245786,False,...,301,60.0,0.273690,False,287.0,NaN,NaN,NaN,NaN,NaN


In [8]:
aggregate_values(data, l)

,m,num_signatures,nist_param_level,threshold,entry_results.0.faulted_coeffs,entry_results.0.total_equations,entry_results.0.filtered_equations,entry_results.0.timeout_limit,entry_results.0.duration,entry_results.0.failure,...,entry_results.4.failure_reason,entry_results.0.failure_reason,entry_results.0.false_positive_rate,entry_results.1.false_positive_rate,entry_results.2.false_positive_rate,entry_results.3.false_positive_rate,entry_results.4.false_positive_rate,duration,false_positive_rate,success
attack_no,,,,,,,,,,,,,,,,,,,,,
0,1270,131,3,196,257,33536,265,60,0.268868,False,...,NaN,NaN,0.030189,0.035714,0.040590,0.033210,0.050725,1.441624,0.038086,True
1,1270,135,3,196,257,34560,266,60,0.199509,False,...,NaN,NaN,0.033835,0.050000,0.044828,0.035842,0.047458,1.353568,0.042392,True
2,1270,135,3,196,279,34560,288,60,0.208690,False,...,NaN,NaN,0.031250,0.046512,0.060071,0.040741,0.047794,6.881954,0.045273,True
3,1271,154,3,196,286,39424,297,60,0.248354,False,...,NaN,NaN,0.037037,0.052117,0.037594,0.059441,0.050167,1.433223,0.047271,True
4,1271,155,3,196,257,39680,266,60,0.240189,False,...,NaN,NaN,0.033835,0.048611,0.045902,0.043046,0.050167,1.619188,0.044312,True
5,1271,153,3,196,280,39168,293,60,0.241885,False,...,NaN,NaN,0.044369,0.088652,0.057823,0.048110,0.062295,11.205268,0.060250,True
6,1272,180,3,196,318,46080,335,60,0.368590,False,...,NaN,NaN,0.050746,0.060703,0.072131,0.043189,0.041045,6.876687,0.053563,True
7,1272,169,3,196,263,43264,280,60,0.311557,False,...,NaN,NaN,0.060714,0.058824,0.051195,0.059211,0.042553,10.133548,0.054499,True
8,1272,169,3,196,271,43264,285,60,0.245786,False,...,NaN,NaN,0.049123,0.034843,0.064748,0.048148,0.046512,9.425587,0.048675,True


In [9]:
data[['num_signatures', 'false_positive_rate', 'success']]

,num_signatures,false_positive_rate,success
attack_no,,,
0,131,0.038086,True
1,135,0.042392,True
2,135,0.045273,True
3,154,0.047271,True
4,155,0.044312,True
5,153,0.060250,True
6,180,0.053563,True
7,169,0.054499,True
8,169,0.048675,True


In [11]:
data_mean = data[['m', 'threshold', 'num_signatures', 'false_positive_rate', 'success']].groupby(['m', 'threshold']).mean()
data_mean

,,num_signatures,false_positive_rate,success
m,threshold,,,
1270,196,133.666667,0.041917,1.000000
1271,196,154.000000,0.050611,1.000000
1272,196,172.666667,0.052246,1.000000
1273,196,199.666667,0.059529,1.000000
1274,196,238.333333,0.070398,0.333333
1275,196,270.333333,0.084806,1.000000
1276,196,334.333333,0.108347,0.333333
1277,196,471.666667,0.138527,0.333333
1278,196,675.000000,0.189797,0.000000


In [ ]:
axes = data_mean[['num_signatures']].plot(kind='bar', legend=False)
axes.set_ylabel("avg. signature count")

In [ ]:
axes = data_mean[['false_positive_rate']].plot(kind='bar', legend=False)
axes.set_ylabel('avg. false-positive rate')

In [ ]:
axes = data_mean[['success']].plot(kind='bar', legend=False)
axes.set_ylabel('avg. success rate')